In [1]:
import time
import mxnet as mx
from mxnet.gluon import nn, Trainer, loss
from mxnet import optimizer
from mxnet import autograd, np, npx, gpu
npx.set_np()
mx.context.num_gpus()

1

In [2]:
gpu = npx.gpu() if npx.num_gpus() > 0 else npx.cpu()


In [2]:
def get_model():
    from tensorflow.keras.models import Sequential
    from tensorflow.keras import initializers
    from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(320, 320, 1), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(320, 320, 1), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(Dropout(0.2))

    model.add(Dense(1, activation='sigmoid',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [3]:

import os
def fullpath(path, files):
    return  [(lambda x: path + x)(x) for x in files]
val_dir = "../chest_xray/chest_xray/train"
filesPneumo = fullpath(val_dir+'/PNEUMONIA/',os.listdir(os.path.join(val_dir, 'PNEUMONIA')))
del filesPneumo[0]
labelsPneumo = np.zeros(len(filesPneumo))
filesNormal = fullpath(val_dir+'/NORMAL/',os.listdir(os.path.join(val_dir, 'NORMAL')))
del filesNormal[0]
labelsNormal = np.ones(len(filesNormal))
fileData = filesPneumo + filesNormal
fileLabels = np.concatenate((labelsPneumo,labelsNormal),axis=0).astype(np.float32)

In [4]:

model = get_model()


In [5]:
from tensorflow.keras.backend import clear_session
def convert_keras_to_mxnet(model):
    def layer_conversion(layer):
        t = str(type(layer)).split(".")
        layer_type = t[-1].replace('\'>',"")
        config = layer.get_config()
        if layer_type == "Dense":
            return nn.Dense(units=config['units'], activation=config['activation'], use_bias=config['use_bias'])
        if layer_type == "Conv2D":
            return nn.Conv2D(channels=config['filters'], kernel_size=config['kernel_size'],use_bias=config['use_bias'],activation=config['activation'], weight_initializer=mx.init.Normal(0.01), bias_initializer=mx.init.Zero())
        if layer_type == "MaxPooling2D":
            return nn.MaxPool2D(pool_size=config['pool_size'])
        if layer_type == "BatchNormalization":
            return nn.BatchNorm(momentum=config['momentum'], epsilon=config['epsilon'])
        if layer_type=="Dropout":
            return nn.Dropout(rate=config['rate'])
        if layer_type=="Flatten":
            return nn.Flatten()
        return None
    mxModel = nn.HybridSequential()
    for layer in model.layers:
        new_layer = layer_conversion(layer)
        mxModel.add(new_layer)
    mxModel.hybridize()
    mxModel.initialize(force_reinit=True, init=mx.init.Xavier(), ctx=mx.cpu(0))
    return mxModel

new_model = convert_keras_to_mxnet(model)
clear_session()
del model

AttributeError: 'Sequential' object has no attribute 'context'

In [34]:
from mxnet import image
def preprocess(img,label):
    img = image.imresize(img/ 255.0,320,320).astype(np.float32)
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
        #img = img[np.newaxis, :].astype(np.float32)
    return img,label
ds2 =  mx.gluon.data.vision.datasets.ImageFolderDataset("../chest_xray/chest_xray/train",transform=preprocess)

/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Ignoring ../chest_xray/chest_xray/train/.DS_Store, which is not a directory.
  
/usr/local/lib/python3.6/dist-packages/mxnet/gluon/data/vision/datasets.py:318: UserWarning: Ignoring ../chest_xray/chest_xray/train/NORMAL/.DS_Store of type . Only support .jpg, .jpeg, .png
  filename, ext, ', '.join(self._exts)))
/usr/local/lib/python3.6/dist-packages/mxnet/gluon/data/vision/datasets.py:318: UserWarning: Ignoring ../chest_xray/chest_xray/train/PNEUMONIA/.DS_Store of type . Only support .jpg, .jpeg, .png
  filename, ext, ', '.join(self._exts)))


In [35]:
train_data = mx.gluon.data.DataLoader(ds2, batch_size=16, shuffle=True)




In [70]:
def acc(output, label):
    # output: (batch, num_output) float32 ndarray
    # label: (batch, ) int32 ndarray
    #pred = x = np.ones((3,4), ctx=mx.gpu(0))
    #pred = pred.reshape(-1,1)
    #for i,o in enumerate(output):
    #    pred[i] = 0. if o < 0.5 else 1.
    o = output.argmax(axis=1)
    ok = 0
    for i,out in enumerate(o):
        if o[i] == label[i].astype('float32'):
            ok += 1
    res = sum(output.argmax(axis=1) == label.astype('float32'))
    return  (ok/len(label))*100


In [73]:
#
#new_model.initialize(force_reinit=True, init=mx.init.Xavier(),ctx=mx.cpu(0))
new_model.collect_params().reset_ctx(ctx=mx.gpu(0))
with mx.Context(mx.gpu(0)):
    accuracy = mx.metric.Accuracy()
    HL = mx.gluon.loss.SigmoidBinaryCrossEntropyLoss()
    trainer = mx.gluon.Trainer(new_model.collect_params(),'sgd', {'learning_rate': 0.0005})
    for epoch in range(30):
        train_loss, train_acc, valid_acc = 0., 0., 0.
        tic = time.time()
        for data, label in train_data:
                            # forward + backward
            with autograd.record():
                output = new_model(data.as_in_ctx(mx.gpu(0)))
                loss = HL(output, label)
            loss.backward()
            trainer.step(16)
                            # calculate training metrics
            train_loss += loss.mean()
            train_acc += acc(output.as_in_ctx(mx.cpu(0)), label.as_in_ctx(mx.cpu(0)))
                        # calculate validation accuracy

            print("Epoch %d: loss %f, train acc %.3f, in %.1f sec" % (epoch, train_loss/len(train_data), train_acc/len(train_data), time.time()-tic))

MXNetError: [05:04:38] src/storage/./pooled_storage_manager.h:161: cudaMalloc retry failed: out of memory
Stack trace:
  [bt] (0) /usr/local/lib/python3.6/dist-packages/mxnet/libmxnet.so(+0x6d554b) [0x7fa0359e054b]
  [bt] (1) /usr/local/lib/python3.6/dist-packages/mxnet/libmxnet.so(+0x41a1692) [0x7fa0394ac692]
  [bt] (2) /usr/local/lib/python3.6/dist-packages/mxnet/libmxnet.so(+0x41a731f) [0x7fa0394b231f]
  [bt] (3) /usr/local/lib/python3.6/dist-packages/mxnet/libmxnet.so(mxnet::NDArray::CheckAndAlloc() const+0x1bd) [0x7fa035a34d5d]
  [bt] (4) /usr/local/lib/python3.6/dist-packages/mxnet/libmxnet.so(void mxnet::CopyFromToDnsImpl<mshadow::gpu, mshadow::cpu>(mxnet::NDArray const&, mxnet::NDArray const&, mxnet::RunContext)+0x3b8) [0x7fa038dead78]
  [bt] (5) /usr/local/lib/python3.6/dist-packages/mxnet/libmxnet.so(void mxnet::CopyFromToImpl<mshadow::gpu, mshadow::cpu>(mxnet::NDArray const&, mxnet::NDArray const&, mxnet::RunContext, std::vector<mxnet::Resource, std::allocator<mxnet::Resourc

In [25]:
mx.context.gpu_memory_info(device_id=0)


(5309792256, 8366784512)